In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score

In [36]:
df = pd.read_csv('https://raw.githubusercontent.com/infoshareacademy/jdsz3-zczasem/ProjektML/ProjektML/Dane/Carseats_Clean.csv')
df.head()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,Urban,US,ShelveLoc_Bad,ShelveLoc_Good,ShelveLoc_Medium
0,9.50,138,73,11,276,120,42,17,1,1,1,0,0
1,11.22,111,48,16,260,83,65,10,1,1,0,1,0
2,10.06,113,35,10,269,80,59,12,1,1,0,0,1
3,7.40,117,100,4,466,97,55,14,1,1,0,0,1
4,4.15,141,64,3,340,128,38,13,1,0,1,0,0


In [37]:
df["Income"].min()

21

In [38]:
df_cls = df
df_cls['Sales_class'] = ''

przedzialy = {}

df_cls['Sales_class'] = np.where(df_cls['Sales'] < df_cls['Sales'].quantile(0.333), 'LOW', np.where(df_cls['Sales'] < df_cls['Sales'].quantile(0.666), 'MED', 'HIGH'))
df['Sales_class_cat'] = df['Sales_class'].factorize()[0]
df.head()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education,Urban,US,ShelveLoc_Bad,ShelveLoc_Good,ShelveLoc_Medium,Sales_class,Sales_class_cat
0,9.50,138,73,11,276,120,42,17,1,1,1,0,0,HIGH,0
1,11.22,111,48,16,260,83,65,10,1,1,0,1,0,HIGH,0
2,10.06,113,35,10,269,80,59,12,1,1,0,0,1,HIGH,0
3,7.40,117,100,4,466,97,55,14,1,1,0,0,1,MED,1
4,4.15,141,64,3,340,128,38,13,1,0,1,0,0,LOW,2


In [39]:
X = df.drop(['Sales','Sales_class','Sales_class_cat'], axis=1).values.astype('float')
y = df['Sales_class_cat'].values

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [41]:
scaler = MinMaxScaler()
scaler.fit(X_train)
norm_x_train = scaler.transform(X_train)
norm_x_test = scaler.transform(X_test)

In [42]:
logreg = LogisticRegression(random_state=123, solver='newton-cg', multi_class='multinomial')
logreg.fit(norm_x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=123, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [43]:
y_pred = logreg.predict(norm_x_test)

In [44]:
metrics.accuracy_score(y_pred, y_test)

0.575

In [45]:
confusion_matrix(y_pred, y_test)

array([[28, 15,  0],
       [ 7, 16, 12],
       [ 3, 14, 25]])

## Dopasowywanie parametrów

In [46]:
grid_search = GridSearchCV(LogisticRegression(),
                          param_grid={'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
                                     'multi_class': ['ovr', 'multinomial']}, cv=5)

grid_search.fit(norm_x_train, y_train)
print(grid_search.best_score_)
print(grid_search.best_params_)

0.6785714285714286
{'multi_class': 'multinomial', 'solver': 'newton-cg'}


In [47]:
y_pred2 = grid_search.best_estimator_.predict(norm_x_test)

In [48]:
confusion_matrix(y_pred2, y_test)

array([[28, 15,  0],
       [ 7, 16, 12],
       [ 3, 14, 25]])

In [49]:
1-((15+7+12+3+14)/(28+15+7+16+12+3+14+25))

0.575